In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats
from scipy.stats.mstats import winsorize

import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10, 6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [ ]:
# function for first looking to the columns

def first_looking(col):
    print("column name    : ", col)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col].isnull().sum())
    print("num_of_uniques : ", df[col].nunique())
    print("--------------------------------")
    print(df[col].value_counts(dropna = False))

In [ ]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
def fill_prop(df, group_col, col_name):
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ", df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
def fill(df, group_col1, group_col2, col_name, method): # method can be either "mode" or "mean" or "median" or "ffill"
    
    '''Fills the missing values with "mode/mean/median/ffill/bfill method" according to double-stage grouping'''
    
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])

    elif method == "mean":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df[col_name].mean(), inplace = True)
        
    elif method == "median":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df[col_name].median(), inplace = True)
        
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [ ]:
df = pd.read_csv("filled_scout.csv")

In [ ]:
df.shape

In [ ]:
df.head(3).T

In [ ]:
first_looking("km")

In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(141)
plt.hist(df.km, bins=50)

plt.subplot(142)
plt.boxplot(df.km)

plt.subplot(143)
plt.boxplot(df.km, whis=2.5)

plt.subplot(144)
plt.boxplot(np.log(df.km), whis=2.5)

plt.show()

In [ ]:
first_looking("Gears")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.Gears, bins=50)

plt.subplot(122)
plt.boxplot(df.Gears, whis=2.5)

plt.show()

In [ ]:
df.Gears.describe()

In [ ]:
df.sort_values(by=["Gears"], ascending=False)["Gears"].head(10)

In [ ]:
df.sort_values(by=["Gears"], ascending=True)["Gears"].head(10)

In [ ]:
df[(df.Gears < 5) | (df.Gears > 8)][["make_model", "body_type", "Gearing_Type", "Gears"]]

In [ ]:
df[(df.make_model == "Renault Espace")][["make_model", "body_type", "Gearing_Type", "Gears"]]

In [ ]:
df["Gears"].replace([1, 2, 3, 4, 9, 50], np.nan, inplace=True)

In [ ]:
df.Gears.isnull().sum()

In [ ]:
fill(df, "make_model", "body_type", "Gears", "mode")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.Gears, bins=50)

plt.subplot(122)
plt.boxplot(df.Gears, whis=1.5)

plt.show()

In [ ]:
first_looking("Displacement_cc")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.Displacement_cc, bins=50)

plt.subplot(122)
plt.boxplot(df.Displacement_cc, whis=2.5)

plt.show()

In [ ]:
df.Displacement_cc.describe()

In [ ]:
df.sort_values(by=["Displacement_cc"], ascending=False)["Displacement_cc"].head(5)

In [ ]:
df.sort_values(by=["Displacement_cc"], ascending=True)["Displacement_cc"].head(55)

In [ ]:
outlier_bool_list = df.Displacement_cc.isin([1, 2, 54, 122, 139, 140, 160, 16000, 15898])

In [ ]:
outlier_bool_list

In [ ]:
df.loc[outlier_bool_list, "Displacement_cc"] = np.nan

In [ ]:
df.Displacement_cc.isnull().sum()

In [ ]:
fill(df, "make_model", "body_type", "Displacement_cc", "mode")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.Displacement_cc, bins=20)

plt.subplot(122)
plt.boxplot(df.Displacement_cc, whis=2.5)

plt.show()

In [ ]:
first_looking("Weight_kg")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.Weight_kg, bins=15)

plt.subplot(122)
plt.boxplot(df.Weight_kg, whis=2.5)

plt.show()

In [ ]:
df.Weight_kg.describe()

In [ ]:
df["Weight_kg"].sort_values().head(10)

In [ ]:
df["Weight_kg"].replace([1, 102], np.nan, inplace = True)

In [ ]:
df["Weight_kg"].isnull().sum()

In [ ]:
fill(df, "make_model", "body_type", "Weight_kg", "mode")

In [ ]:
df["Weight_kg"].isnull().sum()

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.Weight_kg, bins=15)

plt.subplot(122)
plt.boxplot(df.Weight_kg, whis=2.5)

plt.show()

In [ ]:
first_looking("hp_kW")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.hp_kW, bins=15)

plt.subplot(122)
plt.boxplot(df.hp_kW, whis=2.5)

plt.show()

In [ ]:
df.hp_kW.describe()

In [ ]:
df.hp_kW.sort_values().head(50)

In [ ]:
np.sort(df.hp_kW.unique())

In [ ]:
df.hp_kW.loc[df.hp_kW < 40] = np.nan

In [ ]:
df.hp_kW.isnull().sum()

In [ ]:
fill(df, "make_model", "body_type", "hp_kW", "mode")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.hp_kW, bins=15)

plt.subplot(122)
plt.boxplot(df.hp_kW, whis=2.5)

plt.show()

In [ ]:
first_looking("CO2_Emission")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.CO2_Emission, bins=100)

plt.subplot(122)
plt.boxplot(df.CO2_Emission, whis=2.5)

plt.show()

In [ ]:
df.CO2_Emission.describe()

In [ ]:
df.CO2_Emission.value_counts().sort_index()

In [ ]:
df.groupby(["make_model", "body_type", "CO2_Emission"]).price.describe()

In [ ]:
cond = ((df.CO2_Emission < 36) | (df.CO2_Emission > 330))

df.loc[cond, "CO2_Emission"] = np.nan

In [ ]:
df.CO2_Emission.isnull().sum()

In [ ]:
fill(df, "make_model", "body_type", "CO2_Emission", "median")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.CO2_Emission, bins=15)

plt.subplot(122)
plt.boxplot(df.CO2_Emission, whis=2.5)

plt.show()

In [ ]:
plt.figure(figsize = (20, 6))

plt.subplot(121)
plt.hist(np.log(df.CO2_Emission), bins=15)

plt.subplot(122)
plt.boxplot(np.log(df.CO2_Emission), whis=2.5)

plt.show()

In [ ]:
plt.figure(figsize = (20, 6))

plt.subplot(121)
plt.hist(winsorize(df.CO2_Emission, (0.001, 0.003)), bins = 15)

plt.subplot(122)
plt.boxplot(winsorize(df.CO2_Emission, (0.001, 0.003)), whis=2.5)

plt.show()

In [ ]:
first_looking("cons_comb")

In [ ]:
plt.figure(figsize = (20, 6))

plt.subplot(121)
plt.hist(df.cons_comb, bins = 25)

plt.subplot(122)
plt.boxplot(df.cons_comb, whis = 2)

plt.show()

In [ ]:
df.cons_comb.describe()

In [ ]:
df.cons_comb.sort_values().head(10)

In [ ]:
df.cons_comb.sort_values().tail(10)

In [ ]:
first_looking("Nr_of_Doors")

In [ ]:
df.Nr_of_Doors.replace([1, 7], np.nan, inplace=True)

In [ ]:
df.Nr_of_Doors.isnull().sum()

In [ ]:
fill(df, "make_model", "body_type", "Nr_of_Doors", "mode")

In [ ]:
fig, ax = plt.subplots()

df.Nr_of_Doors.value_counts(dropna = False).plot.bar()

for container in ax.containers:
    ax.bar_label(container);

In [ ]:
fig, ax = plt.subplots()

df.groupby(["make_model", "Nr_of_Doors"]).Nr_of_Doors.count().plot.bar()

for container in ax.containers:
    ax.bar_label(container);

In [ ]:
df.drop("Nr_of_Doors", axis=1, inplace=True)

In [ ]:
first_looking("Nr_of_Seats")

In [ ]:
df.Nr_of_Seats.replace([3, 6], np.nan, inplace=True)

In [ ]:
df.Nr_of_Seats.isnull().sum()

In [ ]:
fill(df, "make_model", "body_type", "Nr_of_Seats", "mode")

In [ ]:
fig, ax = plt.subplots()

df.Nr_of_Seats.value_counts(dropna=False).plot.bar()

for container in ax.containers:
    ax.bar_label(container);

In [ ]:
fig, ax = plt.subplots()

df.groupby(["make_model", "Nr_of_Seats"]).Nr_of_Seats.count().plot.bar()

for container in ax.containers:
    ax.bar_label(container);

In [ ]:
df.drop("Nr_of_Seats", axis=1, inplace=True)

In [ ]:
first_looking("price")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.price, bins=20)

plt.subplot(122)
plt.boxplot(df.price, whis=3)

plt.show()

In [ ]:
df.price.describe()

In [ ]:
df.price.sort_values().head(5)

In [ ]:
drop_index = df.price.sort_values().index[:4]
drop_index

In [ ]:
df.drop(drop_index, axis=0, inplace=True)

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.price, bins=20)

plt.subplot(122)
plt.boxplot(df.price, whis=3)

plt.show()

In [ ]:
plt.figure(figsize=(16, 6))
sns.boxplot(x="make_model", y="price", data=df, whis=3)
plt.show()

In [ ]:
first_looking("make_model")

In [ ]:
df[df.make_model=="Audi A2"]

In [ ]:
drop_index = df[df.make_model == "Audi A2"].index
drop_index

In [ ]:
df.drop(drop_index, axis=0, inplace=True)

In [ ]:
df.shape

In [ ]:
first_looking("Fuel")

In [ ]:
df[df.Fuel == "Electric"][["make_model", "body_type", "CO2_Emission", "Gears", "hp_kW", "cons_comb", "price"]]

In [ ]:
drop_index = df[df.Fuel == "Electric"].index
drop_index

In [ ]:
df.drop(drop_index, axis=0, inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.shape

In [ ]:
df.head(3).T

In [ ]:
df.info()

In [ ]:
df_num = df.select_dtypes("number")
df_num

In [ ]:
plt.figure(figsize=(16, 12))

sns.heatmap(df_num.corr(), annot=True, cmap="Blues", linewidths=0.2, annot_kws={"size": 16});

In [ ]:
def color_correlation(val):
    """
    Takes a scalar and returns a string with
    the css property in a variety of color scales 
    for different correlations.
    """
    if val >= 0.6 and val < 0.99999 or val <= -0.6 and val > -0.99999:
        color = 'red'
    elif val < 0.6 and val >= 0.3 or val > -0.6 and val <= -0.3:
        color = 'blue'
    elif val == 1:
        color = 'cyan'    
    else:
        color = 'black'
    return 'color: %s' % color
 
df_num.corr().style.applymap(color_correlation)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.to_csv("final_scout_not_dummy.csv", index=False)

In [ ]:
df = df.join(df["Comfort_Convenience"].str.get_dummies(sep=",").add_prefix("cc_"))
df = df.join(df["Entertainment_Media"].str.get_dummies(sep=",").add_prefix("em_"))
df = df.join(df["Extras"].str.get_dummies(sep = ",").add_prefix("ex_"))
df = df.join(df["Safety_Security"].str.get_dummies(sep=",").add_prefix("ss_"))

In [ ]:
df.drop(["Comfort_Convenience", "Entertainment_Media", "Extras", "Safety_Security"], axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df_final = pd.get_dummies(df, drop_first=True)

In [ ]:
df_final.shape

In [ ]:
df_final.head().T

In [ ]:
df_final.to_csv("final_scout_dummy.csv", index=False)